# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

Automated facial expression recognition provides an objective assessment of emotions. Human based assessment of emotions has many limitations and biases and automated facial expression technology has been found to deliver a better level of insight into behavior patterns. Emotion detection from facial expressions using AI is useful in automatically measuring consumers’ engagement with their content and brands, audience engagement for advertisements, customer satisfaction in the retail sector, psychological analyses, law enforcement etc.

In [ ]:
#@title Explanation Video
from IPython.display import HTML

HTML("""<video width="854" height="480" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Hackathon3b_expression_recognition.mp4" type="video/mp4">
</video>
""")

**Objectives:**

**Stage 4 (15 Marks):** Train a CNN Model and perform Expression Recognition in the EFR Mobile App.

**Stage 5 (5 Marks):** Test for Anti-Face Spoofing on the EFR Mobile App.

##**Stage 4 (15 Marks)**

**(i) Train a CNN Model for Expression Recognition on given Expression data  
(ii) Deploy the Model and Perform Expression Recognition on Team Data through the EFR Mobile App**


---


* Define and train a CNN for expression recognition for the data under folder "Expression_data" which segregated on expression basis.
* Collect your team data using EFR application and test your model on the same and optimize the CNN architecture for predicting the respective labels of the images.
* Save and Download the trained expression model and upload them in the ftp server (refer to [Filezilla Installation and Configuration document](https://drive.google.com/file/d/19UIKpyVK4r12Dxklo8quQdZQ31PWpiKM/view?usp=drive_link)).

* Update the **“exp_recognition.py”** file in the server. Open the files in the terminal (Command prompt) and provide the code for predicting the expression on the face (Note: To define the architecture of your trained model, you'll need to define it in the file **"exp_recognition_model.py"**).

* Test your model on the mobile app for Expression Recognition and Sequence Expression. Your team can also see your results in your terminal.


* Grading Scheme:
> * Expression Recognition (12M): If the functionality is returning expression class correctly for the face using the mobile app’s “Expression Recognition” functionality
> * Sequence Expression (3M): Get three consecutive correct Expressions using the mobile app’s “Sequence Expressions” functionality

**Download the dataset**

In [1]:
#@title Run this cell to download the dataset

from IPython import get_ipython
ipython = get_ipython()

notebook="M3_Hackathon" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Expression_data.zip")

    ipython.magic("sx unzip Expression_data.zip")

    ipython.magic("sx pip install torch==2.5.1 -f https://download.pytorch.org/whl/cu100/stable")
    ipython.magic("sx pip install torchvision==0.11")
    ipython.magic("sx pip install opencv-python")
    print ("Setup completed successfully")
    return
setup()

Setup completed successfully


**Dataset attributes:**

During the setup you have downloaded the Expression data:

* **Expression_data**: In this folder, the images are segregrated in terms of Expression
> * Expressions available: ANGER, DISGUST, FEAR, HAPPINESS, NEUTRAL, SADNESS, SURPRISE
> * Each class is organised as one folder
> * There are ~18000 total images in the training data and ~4500 total images in the testing data

In [2]:
%ls

Expression_data/  Expression_data.zip  __MACOSX/  sample_data/


**Imports: All the imports are defined here**



In [3]:
%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

For the following step, to obtain hints on building a CNN model for face expression, you may refer to this [article](https://drive.google.com/open?id=1P2rpaWW3tOtGGnw4dvtdZ4hjoc8iDNst)

**Define and train a CNN model for expression recognition**

In [4]:
# class ExpressionCNN(nn.Module):
#     def __init__(self):
#         super(ExpressionCNN, self).__init__()
#         # Convolutional Layers
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)

#         # Batch Normalization
#         self.bn1 = nn.BatchNorm2d(32)
#         self.bn2 = nn.BatchNorm2d(64)
#         self.bn3 = nn.BatchNorm2d(128)
#         self.bn4 = nn.BatchNorm2d(256)

#         # Dropout
#         self.dropout = nn.Dropout(0.5)

#         # Fully Connected Layers
#         self.fc1 = nn.Linear(256 * 3 * 3, 512)  # Modified FC1
#         self.fc2 = nn.Linear(512, 256)
#         self.fc3 = nn.Linear(256, 7)  # 7 classes for expressions

#     def forward(self, x):
#         # Convolutional layers with ReLU, BatchNorm, and MaxPooling
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)

#         x = F.relu(self.bn2(self.conv2(x)))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)

#         x = F.relu(self.bn3(self.conv3(x)))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)

#         x = F.relu(self.bn4(self.conv4(x)))
#         x = F.max_pool2d(x, kernel_size=2, stride=2)

#         # Flatten
#         x = x.view(x.size(0), -1)

#         # Fully connected layers with ReLU and Dropout
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)

#         x = F.relu(self.fc2(x))
#         x = self.dropout(x)

#         # Output layer for classification
#         x = self.fc3(x)
#         return x

# # Instantiate and print the model
# model = ExpressionCNN()
# print(model)

In [7]:
class ExpressionCNN(nn.Module):
    def __init__(self):
        super(ExpressionCNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=364, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=364, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=512, out_channels=720, kernel_size=3, stride=1, padding=1)


        # Batch Normalization
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(364)
        self.bn4 = nn.BatchNorm2d(512)
        self.bn5 = nn.BatchNorm2d(720)

        # Dropout
        self.dropout = nn.Dropout(0.5)

        # Fully Connected Layers
        self.fc1 = nn.Linear(720 * 1 * 1, 128)  # Modified FC1
        #self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(128, 7)  # 7 classes for expressions

    def forward(self, x):
        # Convolutional layers with ReLU, BatchNorm, and MaxPooling
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully connected layers with ReLU and Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        #x = F.relu(self.fc2(x))
        #x = self.dropout(x)

        # Output layer for classification
        x = self.fc3(x)
        return x

# Instantiate and print the model
model = ExpressionCNN()
print(model)

ExpressionCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 364, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(364, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 720, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(364, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(720, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=720, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_fea

**Test your model and optimize CNN architecture for predicting the labels correctly**

In [8]:
#   to train CNN model for Expression_Data


# Define the data transformations
transform = transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5,), (0.5,))
])

# Load the dataset
train_dataset = torchvision.datasets.ImageFolder(root='/content/Expression_data/Facial_expression_train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device) # Assuming your model is already defined as 'model'

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
total, correct = 0, 0
num_epochs = 35  # Adjust as needed
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i + 1) % 100 == 0:
            accuracy = 100 * correct / total
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')
            total, correct = 0, 0  # Reset for next batch
    if accuracy >= 90:
        break

print("Training finished.")


Epoch [1/35], Step [100/569], Loss: 1.8158, Accuracy: 26.88%
Epoch [1/35], Step [200/569], Loss: 1.7670, Accuracy: 28.06%
Epoch [1/35], Step [300/569], Loss: 1.7312, Accuracy: 30.31%
Epoch [1/35], Step [400/569], Loss: 1.6919, Accuracy: 28.44%
Epoch [1/35], Step [500/569], Loss: 1.5854, Accuracy: 30.59%
Epoch [2/35], Step [100/569], Loss: 1.7934, Accuracy: 28.71%
Epoch [2/35], Step [200/569], Loss: 1.9218, Accuracy: 28.44%
Epoch [2/35], Step [300/569], Loss: 1.6438, Accuracy: 30.00%
Epoch [2/35], Step [400/569], Loss: 1.8126, Accuracy: 30.16%
Epoch [2/35], Step [500/569], Loss: 1.7983, Accuracy: 31.50%
Epoch [3/35], Step [100/569], Loss: 1.9035, Accuracy: 30.75%
Epoch [3/35], Step [200/569], Loss: 1.7922, Accuracy: 31.38%
Epoch [3/35], Step [300/569], Loss: 1.9528, Accuracy: 31.44%
Epoch [3/35], Step [400/569], Loss: 1.6949, Accuracy: 32.69%
Epoch [3/35], Step [500/569], Loss: 1.6136, Accuracy: 34.66%
Epoch [4/35], Step [100/569], Loss: 1.8031, Accuracy: 33.40%
Epoch [4/35], Step [200/

In [9]:
# Save the model
torch.save(model.state_dict(), 'expression_model.pth')

In [10]:
test_dataset = torchvision.datasets.ImageFolder(root='/content/Expression_data/Facial_expression_test', transform=transform)
test_loader = DataLoader(test_dataset , batch_size=32, shuffle=True)

In [11]:


# Load the saved model
model = ExpressionCNN()  # Assuming ExpressionCNN is defined in your code
model.load_state_dict(torch.load('expression_model.pth'))
model.eval()  # Set the model to evaluation mode

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Testing loop
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculations during testing
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 42.15%


In [ ]:
# YOUR CODE HERE for test evaluation


**Team Data Collection (activate the server first)**

  - (This can be done on the day of the Hackathon once the login username and password are given)

Activate the Server Access
* Open the terminal (Command Prompt)
* Login to SSH by typing **ssh (username)@aiml-sandbox1.talentsprint.com**. Give the login username which is given to you.

Eg: `ssh b16h3gxx@aiml-sandbox1.talentsprint.com`

  (If it is your first time connecting to the server from this computer, accept the connection by typing "yes".)
* After logging into SSH, please activate your virtual environment using the
command **source venv/bin/activate** and then press enter
* You can start the server by giving the command **sh runserver.sh** and then press enter.
* In order to collect team data in mobile app, ensure the server is active


**Collect your team data using the EFR Mobile App and fine-tune the CNN for expression data on your team**

Team Data Collection

* Follow the "Mobile_APP_Documentation" to collect the Expression photos of your team. These will be stored in the server to which login is provided to you.

[Mobile_APP_Documentation](https://drive.google.com/file/d/1F9SU-BwKViK_eZV2-P3pymvGUILUoVFf/view?usp=drive_link)


**Download your team expression data from the EFR app into your colab notebook using the links provided below**

NOTE: Replace the string "username" with your login username (such as b16h3gxx) in the below cell for expression images.

This data will be useful for testing the above trained cnn networks.

In [12]:
!wget -nH --recursive --no-parent --reject 'index.*' https://aiml-sandbox.talentsprint.com/expression_detection/b23h4g19/captured_images_with_Expression/ --cut-dirs=3  -P ./captured_images_with_Expression

--2024-11-16 13:57:23--  https://aiml-sandbox.talentsprint.com/expression_detection/b23h4g19/captured_images_with_Expression/
Resolving aiml-sandbox.talentsprint.com (aiml-sandbox.talentsprint.com)... 139.162.203.12
Connecting to aiml-sandbox.talentsprint.com (aiml-sandbox.talentsprint.com)|139.162.203.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./captured_images_with_Expression/index.html.tmp’

index.html.tmp          [ <=>                ]   1.01K  --.-KB/s    in 0s      

2024-11-16 13:57:24 (225 MB/s) - ‘./captured_images_with_Expression/index.html.tmp’ saved [1038]

Loading robots.txt; please ignore errors.
--2024-11-16 13:57:24--  https://aiml-sandbox.talentsprint.com/robots.txt
Reusing existing connection to aiml-sandbox.talentsprint.com:443.
HTTP request sent, awaiting response... 404 Not Found
2024-11-16 13:57:24 ERROR 404: Not Found.

Removing ./captured_images_with_Expression/index.html.tmp since it should 

In [13]:
%ls

captured_images_with_Expression/  Expression_data.zip   __MACOSX/
Expression_data/                  expression_model.pth  sample_data/


In [14]:
# prompt: provide code for loading the team expression data. Note: Use the same transform which used for Expression_Data

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Use the same transform as used for Expression_Data
transform = transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5,), (0.5,))
])

# Load the team expression data.  Replace 'path/to/team/data'
# with the actual path to your team's image directory.
team_dataset = dset.ImageFolder(root='/content/captured_images_with_Expression', transform=transform)
team_loader = DataLoader(team_dataset, batch_size=32, shuffle=False) # No need to shuffle for testing

# Now you have team_loader, which you can use to iterate over your team's data.
# Example usage:
# for images, labels in team_loader:
#     # Process the images and labels here
#     ...

In [15]:
#
# YOUR CODE HERE for loading the team expression data. Note: Use the same transform which used for Expression_Data.
# YOU CODE HERE for Dataloader: provide code to  team_loader, which you can use to iterate over your team's data.

# Assuming the necessary imports and model definition are already present in the provided code.

# The team_loader is already defined in the provided code:

# team_dataset = dset.ImageFolder(root='/content/captured_images_with_Expression', transform=transform)
# team_loader = DataLoader(team_dataset, batch_size=32, shuffle=False) # No need to shuffle for testing

# Example usage of team_loader (already present in the provided code):
for images, labels in team_loader:
    # Process the images and labels here
    images = images.to(device)  # Move images to the device (GPU or CPU)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)

    # Print or store predictions (example)
    print("Predicted labels:", predicted)
    print("Actual Labels:", labels)

Predicted labels: tensor([0, 4, 4, 4, 0, 0, 4, 2, 5, 0, 0, 0, 0, 0, 4, 4, 0, 1, 5, 3, 5, 3, 0, 0,
        4, 2, 1, 5, 4, 1, 4, 4], device='cuda:0')
Actual Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 2, 2, 2, 2, 2, 2])
Predicted labels: tensor([0, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, 5, 4, 4, 1,
        4, 4, 0, 4, 4, 4, 4, 4], device='cuda:0')
Actual Labels: tensor([2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 5, 5, 5])
Predicted labels: tensor([4, 3, 4, 4, 1, 5, 3, 4, 4, 2, 3, 3, 4, 4, 4, 5, 6, 1, 3, 3, 6, 2],
       device='cuda:0')
Actual Labels: tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])


In [16]:
# prompt: provide code for getting the CNN representation of your team data with expression. Optimize the CNN model for predicting the labels of expressions correctly
# # Note: If the CNN Model is not performing as expected, then you can add your Team Data to the Existing Training Data and Re-Train the Model.

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as dset

# Assuming the necessary imports and model definition are already present in the provided code.
# The model and device are already defined in the preceding code.

# Use the same transform as used for Expression_Data
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

# Load the team expression data.  Replace 'path/to/team/data'
# with the actual path to your team's image directory.
# The path is already corrected in the given code. No change needed here.
team_dataset = dset.ImageFolder(root='/content/captured_images_with_Expression', transform=transform)
team_loader = DataLoader(team_dataset, batch_size=32, shuffle=False) # No need to shuffle for testing


# Get CNN representations and optimize the model:
# Example usage of team_loader to get CNN representations and optimize the model:

# Iterate through the team data
for images, labels in team_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Get CNN representations
    with torch.no_grad(): # We don't need to calculate gradients during inference
        cnn_representations = model.forward(images)
        # You can now use these representations for further analysis or tasks.
        #print(cnn_representations)


    # Optimize the model based on the team data (if needed)
    # Example: if you have additional team data and labels, fine-tune:

    outputs = model(images) # Forward pass
    loss = criterion(outputs, labels)  # Calculate Loss

    optimizer.zero_grad() # Reset optimizer
    loss.backward()         # Backward pass
    optimizer.step()        # Update model parameters


    _, predicted = torch.max(outputs.data, 1)
    print("Predicted labels:", predicted)
    print("Actual Labels:", labels)


# (Optional) Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_expression_model.pth')

Predicted labels: tensor([0, 4, 4, 4, 0, 0, 4, 2, 5, 0, 0, 0, 0, 0, 4, 4, 0, 1, 5, 3, 5, 3, 0, 0,
        4, 2, 1, 5, 4, 1, 4, 4], device='cuda:0')
Actual Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
Predicted labels: tensor([0, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, 5, 4, 4, 1,
        4, 4, 0, 4, 4, 4, 4, 4], device='cuda:0')
Actual Labels: tensor([2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 5, 5, 5], device='cuda:0')
Predicted labels: tensor([4, 3, 4, 4, 1, 5, 3, 4, 4, 2, 3, 3, 4, 4, 4, 5, 6, 1, 3, 3, 6, 2],
       device='cuda:0')
Actual Labels: tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       device='cuda:0')


**Save your trained model**

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the mobile app

 [Hint](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [ ]:
### YOUR CODE HERE for saving the CNN model

In [17]:
# prompt: provide code to save the CNN model

# Save the model
torch.save(model.state_dict(), 'expression_model.pth')

**Download your trained model**
* Given the path of model file the following code downloads it through the browser

In [18]:
from google.colab import files
files.download('/content/expression_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##**Stage 5 (Anti Face Spoofing): (5 marks)**


---



The objective of anti face spoofing is to be able to unlock (say) a screen not just by your image
(which can be easily be spoofed with a photograph of yours) but by a switch in the expression
demanded by the Mobile App (which is much less probable to mimic)
* **Grading scheme**:
> * **Anti Face Spoofing**: (5M Only if both the cases mentioned below are achieved)
>>* **Unlock**: Correct face + Correct Demanded Expression
>>* **Stay Locked**: Correct face + Incorrect Demanded Expression (as you might imagine there are multiple other such possibilities, which you are free to explore)

In [ ]:
# Test in your mobile app and see if it gets unlock.